### Life Coach Application (Yaşam Koçu Uygulaması) [1] <br>

#### Veri Hazirligi
* Özlü söz listeleri olustur (ipucu: split('\n'))
* Listeleri birlestirelim, soz ve kategori listesi olustur
* Sozluk olustur (ipucu: {"Sozler": sozler, "kategori":kategori})
* Listelerden sozluk kullanarak veri cercevesi olustur

#### Islemler
* Rastgele tavsiye
* Secilen kategoride rastgele tavsiye
* Jaccard Benzerligi
$$
benzerlik(A,B) = \frac{|A \cap  B|}{|A \cup  B|}
$$
* Kullanicidan girdi olarak bir cevap al (ipucu: cevap = input("Merhaba insan ne istiyorsun?))
* Kullanici cevabi tek kelimeyse ve bilinen kategorilerden biri ise
    * rastgele bir tavsiye ver
* Kullanici cevabi uzun bir cumleyse ve verimizdeki sozler ile Jaccard benzerligini hesapla
    * ve en yakin sozu dondur

+ ##### Sözcüklerin anlamsal yakınlıklarına göre tavsiyede bulunalım

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
Hayat = """
Olumlu düşünen, bütün olası dünyaların en iyisinde yaşadığımızı söyler ve olumsuz düşünen, bunun gerçek olmasından korkar. James Cabell

Bu dünyaya gelmenin sadece tek bir yolu vardır, terk etmenin ise çok fazla yolu. Donald Harington

Birisinin ‘Hayat zor.’ diye yakındığını duyduğumda, her zaman, ‘Neye kıyasla?’ diye sormayı isterim. Sidney Harris

Mutsuz olmamamız gerekir. Kimsenin hayatla bir kontratı yok. DavidHeath

Hayatını kazanırken yaşamasını bilmeyen bir adam, servetini kazanmadan öncesine göre daha fakirdir. Josiah Holland

Yarının ne olacağını sormaktan vazgeç. Her gün, sana verilen bir hazinedir. Eline geçtikçe değerlendir. Horace

Hayatınızı yaşamanın en iyi yolu, sizden sonra da kalıcı olacak şeyler için harcamaktır. William James

Biz Japonlar, küçük zevklerden hoşlanırız, israftan değil. Daha fazlasını karşılayabilirse de, insanın basit bir yaşam tarzı olması kanısındayım. Massaru Ibuka

Benim sanatım ve mesleğim yaşamaktır. Montaigne

Hayat, çikolata ile dolu bir kutu gibidir. Ne çıkacağını asla bilemezsiniz. Forrest Gump Filmi

Hayat, büyük bir sürprizdir. Ölümün neden daha büyük bir sürpriz olması gerektiğini anlayamıyorum. Vladimir Nabokov
"""

In [ ]:
Sevgi = """
Karım,  benim  ‘Seni seviyorum.’  dememi binlerce kez duydu ama, hiçbir zaman ‘Üzgünüm.’ dediğimi duymadı. Bruce Willis

Büyüklerine saygı, küçüklerine şevkat göstermeyenler, bizden değildir. Hz, Muhammed

Sevip de kaybetmek, sevmemiş olmaktan daha iyidir. Seneca

Sevgiyle düşünün, sevgiyle konuşun, sevgiyle davranın. Her ihtiyaç karşılanacaktır. James Ailen

Sevgi, insanı birliğe, bencillik ise yalnızlığa götürür. Schiller

Sevdiği kadını ve sevdiği işi bulan bir erkek, yeryüzünde cenneti bulmuş demektir. Helen Rowland

Kızgınlık, kırgınlık ya da acıya tutunmayın. Enerjinizi çalarlar ve sizi sevmekten alıkoyarlar. Leo Buscaglia

Herkes temel bir şeyi unutuyor; insanlar sizi, siz onları sevmeden sevmeyecekler. Pat Carroll

Sevgi, dünyadaki yaşamın aktığı nehirdir. Henry Beecher

Sevmeden verebilirsiniz ama, vermeden sevemezsiniz. Amy Carmichael

"""

In [ ]:
Gayret = """
Dileyin verilecektir; arayın bulacaksınız; kapıyı çalın size açılacaktır.    Hz.İsa (a.s.)

Damlayan su, mermeri, yürüyen de dağları deler. Ovidius

Hazine, eziyet çekene gözükür.    Hz. Mevlâna

Beklenen gün gelecekse, çekilen çile kutsaldır.V. Hugo

Yarınlar yorgun ve bezgin kimseler değil, rahatını terk edebilen gayretli insanlara aittir. Cicero

Emek olmadan, yemek olmaz.    Atasözü

Kötülüğün hakim olmaması için tek şart, iyilerin gayret göstermeleridir. Kelâmı Kibar

İnsan bir şeyi çok ciddi olarak arzu etmeye görsün; hiçbir şey erişilemeyecek kadar yükseklerde değildir. H. Andersen

Ümitsizlik ve korkaklık, çalışma azminden yoksun insanlara göredir.  A. Mourois

Bir çığlık, bir çığ meydana getirir.  Soljenitsin

Kıramıyacağm zinciri, hiç olmazsa kemir. C. Şehabettin

Basit bir insanın elinden geleni yapması; bilgili ve zeki bir insanın tembelliğinden çok daha değerlidir.

Taşı delen, suyun kuvveti değil; dalgaların sürekliliğidir.

Damlayan su, taşı deler. Ovidius
"""

In [ ]:
Umut = """
Başlangıçta fazla umut ederiz ama, sonrasında yeteri kadar değil. Joesph Roux

Hayatta umutsuz durumlar yoktur,  sadece umutsuzluk besleyen insanlar vardır. Booth

Şafaktan önce her yer karanlıktır. Katherine Mansfield

Umut, gözle görülemeyeni görür, elle tutulamayanı hisseder ve imkansızı başarır. Anonim

Büyük umutlar, büyük adamlar yaratır. Thomas Fuller

Umut, fakirin ekmeğidir. Thales Miletus

Kazanmayı umut etmeyen, daha baştan kaybetmiştir. Jose Olmedo

Umut etmeyen, hiçbir zaman umutsuzluğa kapılmaz. George Bernard Shaw

Hiçbir kış, sonsuza kadar sürmez. Hal Borland

Umut, şüphe etmek daha kolayken, inanmaya devam etmektir. Anonim

Umut, mutluluktan alınmış bir miktar borçtur. Joseph İoubert
"""

In [ ]:
Idare = """
Çok söyleyen değil, çok iş yapan yöneticiye muhtaçsınız. Hz. Ömer (r.a.)

Sevginin kurduğu devleti, adalet devam ettirir. Farabi

Her memleketin hakettiği bir hükümeti vardır. J. Maistere

Hükümetlerin en kötüsü, suçsuzu korkutandır.

Kendinizi yönetirken kafanızı, başkalarını yönetirken kalbinizi kullanın.

Evi ev eden avrat, yurdu şen eden devlet. Atasözü

Mal cimrilerde, silah korkanlarda, idare de zayıflarda olursa, işler bozulur. Hz. Ebubekir

İnsanlar üç sınıftır: İdareciler, din alimleri ve halk. İdareciler bozulunca, geçim; alimler bozulunca, din; halk bozulunca da hak bozulur. Ebu Bekir Verrak

Kendini yönet, dünyayı yönetecek gücü bulabilirsin. Plato

Kötü idarecilerin başarısı, halkın ıstırabıdır.

Bir memlekette ayaklar baş olursa, başlar ayaklar altında mahvolur.

"""

In [ ]:
categories = [Hayat,Sevgi,Gayret,Umut,Idare]
categories_name = ["hayat", "sevgi", "gayret", "umut", "idare"]

In [8]:
for i in range(len(categories)):
    categories[i] = categories[i].split("\n")


In [9]:
df = pd.DataFrame(columns=['sentence','category'])

In [11]:
for i in range(len(categories_name)):
    for k in range(len(categories[i])):
        df = df.append({'sentence': categories[i][k], 'category': categories_name[i]}, ignore_index=True)
    
    df = df[df['sentence']!=""]
    df.reset_index(drop=True, inplace=True)
    

In [12]:
df

,sentence,category
0,"Olumlu düşünen, bütün olası dünyaların en iyis...",hayat
1,Bu dünyaya gelmenin sadece tek bir yolu vardır...,hayat
2,Birisinin ‘Hayat zor.’ diye yakındığını duyduğ...,hayat
3,Mutsuz olmamamız gerekir. Kimsenin hayatla bir...,hayat
4,Hayatını kazanırken yaşamasını bilmeyen bir ad...,hayat
5,Yarının ne olacağını sormaktan vazgeç. Her gün...,hayat
6,"Hayatınızı yaşamanın en iyi yolu, sizden sonra...",hayat
7,"Biz Japonlar, küçük zevklerden hoşlanırız, isr...",hayat
8,Benim sanatım ve mesleğim yaşamaktır. Montaigne,hayat
9,"Hayat, çikolata ile dolu bir kutu gibidir. Ne ...",hayat


In [13]:
df["cleaned_sentence"]=df['sentence'].copy()

for i in range(len(df)):
    df["cleaned_sentence"][i] = re.sub('[!@#’‘?.,\'$]', '', df["cleaned_sentence"][i])
    df["cleaned_sentence"][i] = df["cleaned_sentence"][i].lower()


### <center>Jaccard Similarity Formula [2]<center>
![Jaccard Similarity Formula](figures/fig_jaccard3.png)

In [14]:
class life_coach:
    def jaccard_similarity(str1, str2):
        a = set(str1.split()) 
        b = set(str2.split())
        intersection = a.intersection(b)
        union = len(a) + len(b) - len(intersection)

        return float(len(intersection) / union)

    def recomendation_random_jaccard():
        jaccard_score_list = []
        answer = input("Hi there, \n What advice would you like? (Hayat/Sevgi/Gayret/Umut/Idare) \n \t Answer: ")
        answer = answer.lower()

        if len(answer.split()) > 1:
            for i in range(len(df)):
                jaccard_score_list.append(life_coach.jaccard_similarity(df["cleaned_sentence"][i], answer))        
                max_index = jaccard_score_list.index(max(jaccard_score_list))
                recommendation = df['sentence'][max_index]

            return recommendation

        else:
            if(answer in categories_name):
                recommendation = df["sentence"][df["category"]==answer].sample(n=1).values[0]

                return recommendation
            else:
                recommendation = df["sentence"].sample(n=1).values[0]
                
            return recommendation
        

In [15]:
life_coach.recomendation_random_jaccard()

Hi there, 
 What advice would you like? (Hayat/Sevgi/Gayret/Umut/Idare) 
 	 Answer: ben nasıl büyük adam olacağım?


'Büyük umutlar, büyük adamlar yaratır. Thomas Fuller'

## <center>Word2Vec by Tomas Mikolov [3, 4] <center>
![Tomas Mikolov](figures/fig_mikolov3.jpg)

In [16]:
start = time.time()

word_vectors = KeyedVectors.load_word2vec_format('trmodel', binary=True)

print("total time: ", time.time() - start)

total time:  7.967896223068237


In [17]:
word_vectors['motivasyon']

array([ 2.68965900e-01, -2.31049150e-01, -1.39877414e-02,  5.01415670e-01,
       -3.87772083e-01,  9.48118344e-02, -3.41180235e-01,  6.61792755e-02,
       -2.00259641e-01, -2.60225773e-01, -1.54859349e-01, -2.94063717e-01,
        8.81631523e-02, -6.93495691e-01,  2.62236565e-01, -7.02954233e-01,
        1.48157239e-01, -2.26100516e-02, -5.08654892e-01, -3.36448401e-01,
       -6.55333698e-01,  6.06544673e-01,  1.45015508e-01,  5.21622837e-01,
        2.30119780e-01,  5.41646898e-01,  1.21688239e-01, -8.31140950e-02,
       -1.70926183e-01,  6.04562238e-02,  4.95102517e-02,  4.68903482e-01,
       -4.21791039e-02,  2.15031639e-01,  1.06118023e-01,  2.20864788e-01,
        3.78131479e-01, -9.36619285e-03, -4.38744090e-02,  2.45283589e-01,
       -2.29311645e-01,  2.98134089e-01, -1.47766873e-01, -6.02133453e-01,
       -2.42539644e-01, -3.93561304e-01,  1.41639158e-01,  2.75545299e-01,
        5.86297549e-02, -2.19686255e-01, -5.91334820e-01,  1.38153091e-01,
       -8.34502801e-02, -

In [19]:
word_vectors.most_similar(positive=["kral","kadın"], negative=["erkek"])

[('kraliçe', 0.508816123008728),
 ('kralı', 0.4718089699745178),
 ('kralın', 0.44512394070625305),
 ('kraliçesi', 0.4190150499343872),
 ('prenses', 0.40713071823120117),
 ('hükümdar', 0.4056030213832855),
 ('prens', 0.3978961706161499),
 ('kraliçenin', 0.39632880687713623),
 ('veliaht', 0.38219934701919556),
 ('tahtı', 0.3773398995399475)]

In [20]:
word_vectors.most_similar(positive=["geliyor","gitmek"],negative=["gelmek"])

[('gidiyor', 0.592621922492981),
 ('gidiyorum', 0.5690363645553589),
 ('gidelim', 0.5637924671173096),
 ('geldim', 0.5413458347320557),
 ('bakıyor', 0.5373592972755432),
 ('gittim', 0.5343414545059204),
 ('gideceğim', 0.5251941680908203),
 ('geldik', 0.5050601363182068),
 ('geliyoruz', 0.5047824382781982),
 ('gider', 0.504159688949585)]

### <center>Cosine Similarity Formula [5]<center>
![Cosine Similarity Formula](figures/fig_cosine_similarity2.png)

In [21]:
class life_coach_w2v:
    def construct_wv_matrices(df, answer):
        word_matrix = np.zeros((len(df), 400))
        for i in range(len(df)):
            wv_sum = np.zeros((1, 400))
            for j in df.iloc[i]['cleaned_sentence'].split(' '):
                try:
                    wv_sum += word_vectors[j]
                except: 
                    pass
            word_matrix[i] = np.divide(wv_sum, len(df.iloc[i]['cleaned_sentence'].split(' ')))

            answer_vector = np.zeros((1, 400))
            for k in answer.split(' '):
                try:
                    answer_vector += word_vectors[k]
                except: 
                    pass
            answer_vector = np.divide(answer_vector, len(answer.split(' ')))

        return word_matrix, answer_vector

    def recommendation_wv():
        answer = input("Hi there, \n What advice would you like? (Hayat/Sevgi/Gayret/Umut/Idare) \n \t Answer: ")
        answer = answer.lower()

        word_matrix, answer_vector = life_coach_w2v.construct_wv_matrices(df, answer)

        score_board = []
        for i in range(len(df)):
            score = cosine_similarity(word_matrix[i].reshape(1,400), answer_vector.reshape(1,400))
            score_board.append(score)
        recommendation = df.iloc[score_board.index(max(score_board))].sentence
        
        return recommendation
    

In [22]:
life_coach_w2v.recommendation_wv()

Hi there, 
 What advice would you like? (Hayat/Sevgi/Gayret/Umut/Idare) 
 	 Answer: ben nasıl büyük adam olacağım?


'Büyük umutlar, büyük adamlar yaratır. Thomas Fuller'

### References 

[1] https://github.com/uzay00/KaVe-Egitim/tree/master/VeriBilimi <br>
[2] http://tiny.cc/cp1o4y <br>
[3] Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." Advances in neural information processing systems. 2013. <br>
[4] Mikolov, Tomas, et al. "Efficient estimation of word representations in vector space." arXiv preprint arXiv:1301.3781 (2013). <br>
[5] https://github.com/akoksal/Turkish-Word2Vec <br>
[6] http://tiny.cc/iq1o4y